In [ ]:
# Introduction to the BeautifulSoup library.
from bs4 import BeautifulSoup
# import lxml as subsequences if the html parser is not work.

with open("/Users/ez/OneDrive/Máy tính/codeprojects/PYTHON/100_days_of_Py/10_day(45-50)/b-soup/website.html", encoding="utf8") as file:
    contents = file.read()                                                                                    # Fix the UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 281: character maps to <undefined> bug.

soup = BeautifulSoup(contents, "html.parser") # lxml

# print(soup.title)   
# print(soup.title.name) # Name of the tag which is 'title'.
# print(soup.title.string) # Content of title tag.
print(soup.prettify()) # Indented format all the content of the html file.
print(soup.p) # Or <a>, <li>,... tag.

#----------------------------------------------------------#

all_anchor_tag = soup.find_all(name="a") # Print out all anchors tag.
print(all_anchor_tag)
for tag in all_anchor_tag:
    # print(tag.getText())
    print(tag.get("href")) # Print out the links contained in the anchor tags.

#----------------------------------------------------------#

heading = soup.find(name="h1", id="name")
print(heading)

section_heading = soup.find(name="h3", class_="heading") # Cannot use the word class because it is a keyword in python.
print(section_heading)

company_url = soup.select_one(selector="p a") # To select a specific anchor tag using CSS selector.
print(company_url)

name = soup.select_one("#name") # ID
print(name)

headings = soup.select(".heading") # Class 
print(headings)

#----------------------------------------------------------#

form_tag = soup.find("input")
max_length = form_tag.get("maxlength")
print(max_length) # Get the maxlength value

#----------------------------------------------------------#

In [ ]:
# 🚨 Scraping a Live Website
from bs4 import BeautifulSoup
import requests

response = requests.get("https://news.ycombinator.com/news")
yc_we_page = response.text

soup = BeautifulSoup(yc_we_page, "html.parser")
articles = soup.find_all(name="a", class_="titlelink")
article_texts = []
article_links = []
for article in articles:
    text = article.getText()
    article_texts.append(text)
    link = article.get("href")
    article_links.append(link)
article_upvote = [int(score.getText().split(" ")[0]) for score in soup.find_all(name="span", class_="score")]

max_points_index = article_upvote.index(max(article_upvote))
print(article_texts[max_points_index])
print(article_links[max_points_index])

In [ ]:
# 🚨 Go to the "https//.../robots.txt" to read the ethical codes of conduct before scraping the website.

In [ ]:
# 🚨 100 Movies to watch.
import requests
from bs4 import BeautifulSoup

URL = "https://web.archive.org/web/20200518073855/https://www.empireonline.com/movies/features/best-movies-2/"

response = requests.get(URL)
the_100_mv_page = response.text

soup = BeautifulSoup(the_100_mv_page, "html.parser")
movies_list = soup.find_all(name="h3", class_="title")
movies_list_reverse = movies_list.reverse()

with open("/Users/ez/OneDrive/Máy tính/codeprojects/PYTHON/100_days_of_Py/10_day(45-50)/100-movies/list.txt", mode="w", encoding="utf-8") as f:
    for movie in movies_list:
        f.write(f"{movie.getText()}\n")

In [ ]:
# 🚨 100 Movies to watch solution.
import requests
from bs4 import BeautifulSoup

URL = "https://web.archive.org/web/20200518073855/https://www.empireonline.com/movies/features/best-movies-2/"

# Write your code below this line 👇

response = requests.get(URL)
website_html = response.text

soup = BeautifulSoup(website_html, "html.parser")

all_movies = soup.find_all(name="h3", class_="title")

movie_titles = [movie.getText() for movie in all_movies]
movies = movie_titles[::-1]

with open("movies.txt", mode="w") as file:
    for movie in movies:
        file.write(f"{movie}\n")

In [ ]:
# 🚨 Sportify playlist generator of top 100 Billboard Music any day.
import requests
from bs4 import BeautifulSoup
import spotipy
from spotipy import oauth2

SPOTIPY_CLIENT_ID = 'Your key'
SPOTIPY_CLIENT_SECRET = 'Your secret key'
SPOTIPY_REDIRECT_URI = 'http://example.com'

sp_oauth = oauth2.SpotifyOAuth(
    SPOTIPY_CLIENT_ID, 
    SPOTIPY_CLIENT_SECRET,
    SPOTIPY_REDIRECT_URI, 
    scope="playlist-modify-private", 
    show_dialog=True,
    # cache_path="/Users/ez/OneDrive/Máy tính/codeprojects/PYTHON/100_days_of_Py/10_day(45-50)/sportify-playlist/.cache"
    )

#------------------------ Get the access token ------------------------#

# ad = sp_oauth.get_access_token(as_dict=False)
# print(ad)

#------------------------ Get the access token ------------------------#

header = {
    "Authorization": "Bearer your_key"
}

sp = spotipy.Spotify(auth=header, oauth_manager=sp_oauth)
user_id = sp.current_user()["id"]
scrap_day = input("Which year do you want to travel to? Type the date in this format YYYY-MM-DD: ")

#------------------------ get 100 billboard songs of the day ------------------------#

URL = f"https://www.billboard.com/charts/hot-100/{scrap_day}/"

response = requests.get(URL)
website_html = response.text

soup = BeautifulSoup(website_html, "html.parser")

song_titles = soup.select(selector="li #title-of-a-story")
with open("/Users/ez/OneDrive/Máy tính/codeprojects/PYTHON/100_days_of_Py/10_day(45-50)/sportify-playlist/songs.txt", mode="w", encoding="utf-8") as f:
    for song in song_titles:
        f.write(f"{song.getText().strip()}\n")

#------------------------ add songs from the list_song to the playlist ------------------------#

with open("/Users/ez/OneDrive/Máy tính/codeprojects/PYTHON/100_days_of_Py/10_day(45-50)/sportify-playlist/songs.txt", mode="r", encoding="utf-8") as f:
    song_names = f.read().splitlines() # Readlines without endline

song_uris = []
year = scrap_day.split("-")[0]
for song in song_names:
    result = sp.search(q=f"track:{song} year:{year}", type="track")
    print(result)
    try:
        uri = result["tracks"]["items"][0]["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song} doesn't exist in Spotify. Skipped.")
playlist = sp.user_playlist_create(user=user_id, name=f"{scrap_day} Billboard 100", public=False)

sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

In [ ]:
# 🚨 Automated Amazon price tracker.
from bs4 import BeautifulSoup
import requests
import smtplib

my_email = "nguyennam2741@yahoo.com"
my_password = "G-5*EqpY!mLrijH"

URL = f"https://www.amazon.com/Kindle-Paperwhite-Essentials-Bundle-including/dp/B09FBXR5Q8/ref=sr_1_3?crid=1KAVVINZLWW7F&keywords=kindle&qid=1659596710&sprefix=kindl%2Caps%2C713&sr=8-3"

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36",
    "Accept-Language": "vi-VN,vi;q=0.9,fr-FR;q=0.8,fr;q=0.7,en-US;q=0.6,en;q=0.5",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"
}

response = requests.get(URL, headers=header)
website_html = response.text

soup = BeautifulSoup(website_html, "html.parser")#.encode("utf-8")
price = soup.find(name="span", class_="a-offscreen").getText().split("$")
if float(price[1]) < 150:
    with smtplib.SMTP("smtp.mail.yahoo.com") as connection:
        connection.starttls()
        connection.login(user=my_email, password=my_password)
        connection.sendmail(from_addr=my_email, to_addrs="ntguppy13@gmail.com", msg="HOT HOT\n\nThe price had dropped below your buy price!")

In [ ]:
selenium_doc_url = "https://selenium-python.readthedocs.io/locating-elements.html"

In [ ]:
# 🚨 Python.org events list.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.python.org/")

events = {}

list_length = len(driver.find_elements(By.XPATH, '//*[@id="content"]/div/section/div[2]/div[2]/div/ul/li')) 
# Can copy the xpath by right click on the element and choose copy then Xpath
# Use the get_attribute("...") to get the thing you want.
for i in range(1, list_length+1):
    time = driver.find_element(By.XPATH, f'//*[@id="content"]/div/section/div[2]/div[2]/div/ul/li[{i}]/time').text
    name = driver.find_element(By.XPATH, f'//*[@id="content"]/div/section/div[2]/div[2]/div/ul/li[{i}]/a').text
    events[i] = {
        "time": f"{time}",
        "name": f"{name}",
    }

print(events)

In [ ]:
# 🚨 Wikipedia articles.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://en.wikipedia.org/wiki/Main_Page/")

article_cnt = driver.find_element(By.XPATH, '//*[@id="articlecount"]/a[1]')
print(article_cnt.text)

In [ ]:
# 🚨 Form filling exercise
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("http://secure-retreat-92358.herokuapp.com/")

first_name = driver.find_element(By.XPATH, '/html/body/form/input[1]')
first_name.send_keys("Thanh")

last_name = driver.find_element(By.XPATH, '/html/body/form/input[2]')
last_name.send_keys("Nguyen")

email = driver.find_element(By.XPATH, '/html/body/form/input[3]')
email.send_keys("ThanhNguyen@gmail.com")

button = driver.find_element(By.XPATH, '/html/body/form/button')
button.click()

In [ ]:
# 🚨 Auto CookieClicker play bot.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import StaleElementReferenceException

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("http://orteil.dashnet.org/experiments/cookie/") 

continue_game = 1
while continue_game == 1:
    try:
        cookie_clicker = driver.find_element(By.XPATH, '//*[@id="cookie"]')
        cookie_clicker.click()

        crursor = driver.find_element(By.XPATH, '//*[@id="buyCursor"]/b')
        grandma = driver.find_element(By.XPATH, '//*[@id="buyGrandma"]/b')
        factory = driver.find_element(By.XPATH, '//*[@id="buyFactory"]/b')
        mine = driver.find_element(By.XPATH, '//*[@id="buyMine"]/b')
        shipment = driver.find_element(By.XPATH, '//*[@id="buyShipment"]/b')
        alchemy_lab = driver.find_element(By.XPATH, '//*[@id="buyAlchemy lab"]/b')
        portal = driver.find_element(By.XPATH, '//*[@id="buyPortal"]/b')
        time_machine = driver.find_element(By.XPATH, '//*[@id="buyTime machine"]/b')
        upgrade_list = [crursor, grandma, factory, mine, shipment, alchemy_lab, portal, time_machine]
        money_list = [int(n.text.split(" - ")[1].replace(",","")) for n in upgrade_list]
        money = int(driver.find_element(By.XPATH, '//*[@id="money"]').text)

        if money > min(money_list):
            i = money_list.index(min(money_list))
            upgrade_list[i].click()
    except StaleElementReferenceException:
        continue

In [ ]:
# 🚨 LinkedIn auto job save.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.linkedin.com/jobs/search/?f_LF=f_AL&geoId=102257491&keywords=python%20developer&location=London%2C%20England%2C%20United%20Kingdom&redirect=false&position=1&pageNum=0")

sign_in = driver.find_element(By.XPATH, '/html/body/div[1]/header/nav/div/a[2]')
sign_in.click()
email_sign_in = driver.find_element(By.XPATH, '//*[@id="username"]')
email_sign_in.send_keys('********************')
password_sign_in = driver.find_element(By.XPATH, '//*[@id="password"]')
password_sign_in.send_keys('***************')
sign_in_button = driver.find_element(By.XPATH, '//*[@id="organic-div"]/form/div[3]/button')
sign_in_button.click()
job_list = driver.find_elements(By.CSS_SELECTOR, "a .disabled ember-view job-card-container__link job-card-list__title")
for job in job_list:
    try:
        time.sleep(2)
        job.click()
        time.sleep(2)
        save_button = driver.find_element(By.CSS_SELECTOR, "button .jobs-save-button artdeco-button artdeco-button--3 artdeco-button--secondary")
        save_button.click()
    except NoSuchElementException:
        print("No save button, skipped.")
time.sleep(5000)

In [ ]:
# 🚨 Auto Tinder swiping bot.
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import time 

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://tinder.com/")
time.sleep(2)
log_in = driver.find_element(By.XPATH, '//*[@id="q554704800"]/div/div[1]/div/main/div[1]/div/div/div/div/header/div/div[2]/div[2]/a')
log_in.click()
time.sleep(2)
more_opt = driver.find_element(By.XPATH, '//*[@id="q-1173676276"]/div/div/div[1]/div/div/div[3]/span/button')
more_opt.click()
time.sleep(2)
log_in_gg = driver.find_element(By.XPATH, '//*[@id="q-1173676276"]/div/div/div[1]/div/div/div[3]/span/div[2]/button/span[2]')
log_in_gg.click()
base_window = driver.window_handles[0]
fb_login_window = driver.window_handles[1]
driver.switch_to.window(fb_login_window)
time.sleep(2)
email_address = driver.find_element(By.XPATH, '//*[@id="email"]')
email_address.send_keys("Your email")
time.sleep(2)
password = driver.find_element(By.XPATH, '//*[@id="pass"]')
password.click()
password.send_keys("Your password")
password.send_keys(Keys.ENTER)
driver.switch_to.window(base_window)
time.sleep(10)
allow_locate = driver.find_element(By.XPATH, '//*[@id="q-1173676276"]/div/div/div/div/div[3]/button[1]/span')
allow_locate.click()
time.sleep(2)
noti_enable = driver.find_element(By.XPATH, '//*[@id="q-1173676276"]/div/div/div/div/div[3]/button[1]/span')
noti_enable.click()
time.sleep(2)
cookie_acpt = driver.find_element(By.XPATH, '//*[@id="q554704800"]/div/div[2]/div/div/div[1]/div[1]/button/span')
cookie_acpt.click()
time.sleep(10)
for i in range(100):
    try:
        like_button = driver.find_element(By.XPATH, '//*[@id="q554704800"]/div/div[1]/div/main/div[1]/div/div')
        like_button.send_keys(Keys.ARROW_LEFT)
    except NoSuchElementException:
        continue